In [1]:
import sys
import pickle
import pandas as pd
from collections import Counter
from collections import defaultdict
import pyarrow
from matplotlib import pyplot as plt
import seaborn
import re

In [ ]:
#/wynton/group/ahituv/MPRAflow/src/nf_filter_barcodes.py

In [25]:
PREF = "null_mpra" 
ENH_BC = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/null_mpra_coords_to_barcodes.pickle"
COUNT_TABLE = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/null_mpra_barcodes_per_candidate-no_repeats-no_jackpots.feather" 
COV_THRESH = 3 
COV_FRAC= 0.5 
LABEL_FILE = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc/label_rmIllegalChars.txt"
label=pd.read_csv(LABEL_FILE,header='infer',sep='\t')
label.head()

,endog|HepG2_DNasePeakNoPromoter10535|active,na
0,endog|HepG2_DNasePeakNoPromoter10586|active,na
1,endog|HepG2_DNasePeakNoPromoter10714|inactive,na
2,endog|HepG2_DNasePeakNoPromoter10849|inactive,na
3,endog|HepG2_DNasePeakNoPromoter11263|active,na
4,endog|HepG2_DNasePeakNoPromoter11270|active,na


In [ ]:
PREF=sys.argv[1]
ENH_BC=sys.argv[2]
COUNT_TABLE=sys.argv[3]

In [ ]:
PREF=sys.argv[1]
ENH_BC=sys.argv[2]
COUNT_TABLE=sys.argv[3]


lab_opt="TRUE"

#ENH_BC=PREF+'_coords_to_barcodes.pickle'
#COUNT_TABLE=PREF+'_barcodes_per_candidate-no_repeats-no_jackpots.feather'

COV_THRESH=sys.argv[4]
COV_FRAC=sys.argv[5]
LABEL_FILE=pd.read_csv(sys.argv[6],header='infer',sep='\t')

In [5]:
label.columns=['coord','label']
#print(LABEL_FILE.head())
#print(ENH_BC)
#print(COUNT_TABLE)

#function to get rid of lowly covered BCs and get rid of BCs that map to multiple other enhancers
def covered_no_perm(min_threshold,min_frac, mydict):
    out_dict={}
    #count coverage of all bcs
    all_bcs=(mydict.values())
    flat_BC = [item for sublist in all_bcs for item in sublist]
    counts=Counter(flat_BC)
    
    #filter bcs that are not mapping in the majority to the insert and do not meet the minimum coverage in that insert
    for k,v in mydict.items():
        bc_counts=(Counter((v)))
        new_v=[]
        for i in v:
            #print(i)
            if(bc_counts[i]>=int(min_threshold)):
                #print(bc_counts[i])
                per=float((bc_counts[i]/counts[i]))
                if(per>=float(min_frac)):
                    #print(per)
                    new_v.append(i)
        out_dict[k]=set(new_v) 

    return(out_dict)


    # for k,v in mydict.items():
    #     bc_counts=(Counter(v))
    #     filtered=({x : bc_counts[x] for x in bc_counts if bc_counts[x] >= int(min_threshold) and bc_counts[x]<= int(50)})
    #     new_BCs=(list(filtered.keys()))
    #     mydict[k]=new_BCs

    ##get rid of BCs that are premiscuous 
    #all_bcs=(mydict.values())
    #flat_BC = [item for sublist in all_bcs for item in sublist]
    #counts=Counter(flat_BC)
    #permis=({x : counts[x] for x in counts if counts[x] > 1})
    #dups=(list(permis.keys()))
    #print(dups)

    ##remove duplicates and get rid of poly A/G/T/C (7 in a row maybe change to more)
    #for k,v in mydict.items():
    #    new_v=[]
    #    #print(k,v)
    #    for i in v:
    #        #print(i)
    #        if i not in dups:
    #            m=0
    #            for match in re.finditer(r"(\w)\1\1\1\1\1\1", i):   
    #                #print('loop')
    #                m=i[match.start():match.end()]
    #                #print(m)
    #            if(m==0):
    #                new_v.append(i)
    #    out_dict[k]=new_v
    #return(out_dict)



#takes in a pandas series count dataframe and plots the coverage coloring by type of control sequence
def create_lib_plots(c_obj, plot_name, sum_name,lab_opt="FALSE", label_frame):
    #change counter to dataframe
    df=c_obj
    #df = pd.DataFrame.from_dict(c_obj,orient='index').reset_index()
    #print(df)
    lab_opt="TRUE"
    #add label column for 'class'
    if lab_opt =="TRUE":
        
        #label=[]
        #for i in df.coord:
        #    tmp=(str.split(str(i),'_C:')[0])
        #    label.append(str(str.split(str(tmp),'_r')[0]))
        labels=label_frame
        df=pd.merge(df,labels,on='coord')
        #df['label']=label
        df=df.sort_values(['label','n_barcodes'])
        print('sorted df')
        print(df.head())
        #plot
        #a4_dims = (11.7, 8.27)
        plt.figure(figsize=(16, 6))
        p=seaborn.violinplot(data=df,x="label", y="n_barcodes",dodge=False)
        #p=seaborn.barplot(data=df,x="coord", y="n_barcodes",hue="label",dodge=False)
        print('name')
        print(plot_name)
        #p.set(xticklabels=[])
        fig=p.get_figure()
        fig.savefig(plot_name)
    else:
        plt.figure(figsize=(16, 6))
        violinplot
        p=seaborn.violinplot(data=df,x="label", y="n_barcodes",dodge=False)
        #p=seaborn.barplot(data=df,x="coord", y="n_barcodes",dodge=False)
        #p.set(xticklabels=[])
        fig=p.get_figure()
        fig.savefig(plot_name)

    print('desc')
    print(df.describe)
    #summary=pd.DataFrame(df.describe)
    #summary.to_csv(sum_name, index=False,)
    df.describe().to_csv(sum_name)

In [7]:
#plot original data
create_lib_plots(pd.read_feather(COUNT_TABLE),PREF+"_original_counts.png","original_count_summary.txt","TRUE")

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed

# create label fram

In [14]:
c_obj = pd.read_feather(COUNT_TABLE)
plot_name= PREF+"_original_counts.png"
sum_name="original_count_summary.txt"
lab_opt="FALSE"
#change counter to dataframe
df=c_obj
#df = pd.DataFrame.from_dict(c_obj,orient='index').reset_index()
#print(df)
lab_opt="TRUE"
#add label column for 'class'
## #if lab_opt =="TRUE":

In [15]:
df.head()

,coord,n_barcodes
0,endog|HepG2_DNasePeakNoPromoter10535|active,1977206


In [16]:
    #label=[]
    #for i in df.coord:
    #    tmp=(str.split(str(i),'_C:')[0])
    #    label.append(str(str.split(str(tmp),'_r')[0]))
    labels=label
    

In [19]:
print(label.shape)

(109524, 2)


In [21]:
label.head()

,coord,label
0,endog|HepG2_DNasePeakNoPromoter10586|active,na
1,endog|HepG2_DNasePeakNoPromoter10714|inactive,na
2,endog|HepG2_DNasePeakNoPromoter10849|inactive,na
3,endog|HepG2_DNasePeakNoPromoter11263|active,na
4,endog|HepG2_DNasePeakNoPromoter11270|active,na


In [22]:
label.loc[label["label"]!="na"]

,coord,label


In [11]:
    df=pd.merge(df,labels,on='coord')
    #df['label']=label
    df=df.sort_values(['label','n_barcodes'])
    print('sorted df')
    print(df.head())

sorted df
Empty DataFrame
Columns: [coord, n_barcodes, label]
Index: []


In [9]:
    #plot
    #a4_dims = (11.7, 8.27)
    plt.figure(figsize=(16, 6))
    p=seaborn.violinplot(data=df,x="label", y="n_barcodes",dodge=False)
    #p=seaborn.barplot(data=df,x="coord", y="n_barcodes",hue="label",dodge=False)
    print('name')
    print(plot_name)
    #p.set(xticklabels=[])
    fig=p.get_figure()
    fig.savefig(plot_name)
else:
    plt.figure(figsize=(16, 6))
    violinplot
    p=seaborn.violinplot(data=df,x="label", y="n_barcodes",dodge=False)
    #p=seaborn.barplot(data=df,x="coord", y="n_barcodes",dodge=False)
    #p.set(xticklabels=[])
    fig=p.get_figure()
    fig.savefig(plot_name)

print('desc')
print(df.describe)
#summary=pd.DataFrame(df.describe)
#summary.to_csv(sum_name, index=False,)
df.describe().to_csv(sum_name)

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed

In [23]:
sum_name

'original_count_summary.txt'

In [24]:
#filter and plot new data
dic=pickle.load(open(ENH_BC,'rb'))
filtered_dict=covered_no_perm(COV_THRESH,COV_FRAC, dic)

#new counts
filtered_counts=(pd.Series(filtered_dict, name = 'n_barcodes').str.len().rename_axis('coord').reset_index())
print('filter')
print(filtered_counts)
create_lib_plots(filtered_counts, PREF+"_filtered_counts.png","filtered_count_summary.txt","FALSE")
pickle.dump(filtered_dict,open(PREF+"_filtered_coords_to_barcodes.pickle","wb"))

filter
                                         coord  n_barcodes
0  endog|HepG2_DNasePeakNoPromoter10535|active       70793


TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed